In [1]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/dw_matrix_road_sign


In [2]:
!pwd


/content/drive/My Drive/Colab Notebooks/dw_matrix_road_sign


In [5]:
!pip install hyperopt

In [23]:
from google.colab import drive
import os
import datetime
import pandas as pd
import numpy as np
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from skimage import color, exposure
%load_ext tensorboard

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [0]:
train=pd.read_pickle('data/train.p')
test=pd.read_pickle('data/test.p')
signnames=pd.read_csv('data/signnames.csv')
labels_dict=signnames.to_dict()['b']

X_train, y_train=train['features'], train['labels']
X_test, y_test=test['features'], test['labels']

if y_train.ndim==1: y_train=to_categorical(y_train)
if y_test.ndim==1: y_test=to_categorical(y_test)

input_shape=X_train.shape[1:]
num_classes=y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, param_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  logdir=os.path.join('logs',datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))  
  tensorboard_callback=tf.keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=1)
  model.fit(
      X_train, 
      y_train,
      batch_size=param_fit.get('batch_size',128),
      epochs=param_fit.get('epochs', 5),
      verbose=param_fit.get('verbose',1),
      validation_data=param_fit.get('validation_data',(X_train, y_train)),
      callbacks=[tensorboard_callback]

      )
  return model

In [0]:
def predict(model_trained, X_test, y_test, scoring=accuracy_score):
  y_pred_prob=model_trained.predict(X_test)
  y_pred=np.argmax(y_pred_prob, axis=1)
  y_test_norm = np.argmax(y_test, axis=1)
  return scoring(y_test_norm, y_pred)

In [10]:
def get_cnn_v4(input_shape, num_classes):
  return Sequential([
                  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape, padding='same'),
                  Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
                  MaxPool2D(),
                  Dropout(0.5),
             
                  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
                  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
                  MaxPool2D(),
                  Dropout(0.25),
                  Flatten(),

                  Dense(1024, activation='relu'),
                  Dropout(0.25),
                  Dense(1024, activation='relu'),
                  Dropout(0.25),
                  Dense(num_classes, activation='softmax')
        ])
get_cnn_v4((32,32,1), num_classes).summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 64)          0

In [0]:
param_fit2={
    'epochs': 5
}

In [0]:
def train_and_predict2(model):
  model_trained=train_model(model, X_train, y_train, param_fit=param_fit2)
  return predict(model_trained, X_test,y_test)

In [18]:
train_and_predict2(get_cnn_v4(input_shape, num_classes))

Epoch 1/5
272/272 [==============================] - 6s 20ms/step - loss: 3.1986 - accuracy: 0.3025 - val_loss: 1.1175 - val_accuracy: 0.6812
Epoch 2/5
272/272 [==============================] - 5s 19ms/step - loss: 0.9911 - accuracy: 0.6936 - val_loss: 0.4803 - val_accuracy: 0.8598
Epoch 3/5
272/272 [==============================] - 5s 20ms/step - loss: 0.6811 - accuracy: 0.7851 - val_loss: 0.3370 - val_accuracy: 0.9024
Epoch 4/5
272/272 [==============================] - 5s 20ms/step - loss: 0.5038 - accuracy: 0.8403 - val_loss: 0.1779 - val_accuracy: 0.9490
Epoch 5/5
272/272 [==============================] - 5s 20ms/step - loss: 0.4189 - accuracy: 0.8684 - val_loss: 0.1743 - val_accuracy: 0.9456


0.8476190476190476

In [0]:
def func_obj(param):
  model=get_model(param)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  model.fit(
      X_train, 
      y_train,
      batch_size=int(param.get('batch_size',128)),
      epochs=5,
      verbose=0
      )
  
  score=model.evaluate(X_test, y_test, verbose=0)
  accuracy=score[1]
  print(param, 'accuracy={}'.format(accuracy))

  return{'loss': -accuracy, 'status': STATUS_OK, 'model':model}

In [0]:
def get_model(params):
  return Sequential([
                  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape, padding='same'),
                  Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
                  MaxPool2D(),
                  Dropout(float(params['dropout_cnn_block_1'])),
             
                  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
                  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
                  MaxPool2D(),
                  Dropout(float(params['dropout_cnn_block_2'])),

                  Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
                  Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
                  MaxPool2D(),
                  Dropout(float(params['dropout_cnn_block_3'])),
                  Flatten(),

                  Dense(1024, activation='relu'),
                  Dropout(float(params['dropout_cnn_block_4'])),
                  Dense(1024, activation='relu'),
                  Dropout(float(params['dropout_cnn_block_5'])),
                  Dense(num_classes, activation='softmax')
        ])
#get_model(space).summary()

In [0]:
space={
    'batch_size': hp.quniform('batch_size', 30, 200, 10),
    'dropout_cnn_block_1': hp.quniform('dropout_cnn_block_1', 0.3, .5,.1),
    'dropout_cnn_block_2': hp.quniform('dropout_cnn_block_2', 0.3, .5,.1),
    'dropout_cnn_block_3': hp.quniform('dropout_cnn_block_3', 0.3, .5,.1),
    'dropout_cnn_block_4': hp.quniform('dropout_cnn_block_4', 0.3, .7,.1),
    'dropout_cnn_block_5': hp.quniform('dropout_cnn_block_5', 0.3, .7,.1)
}

In [66]:
best_min=fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)
    

{'batch_size': 130.0, 'dropout_cnn_block_1': 0.30000000000000004, 'dropout_cnn_block_2': 0.4, 'dropout_cnn_block_3': 0.30000000000000004, 'dropout_cnn_block_4': 0.5, 'dropout_cnn_block_5': 0.7000000000000001}
accuracy=0.9548752903938293
{'batch_size': 90.0, 'dropout_cnn_block_1': 0.5, 'dropout_cnn_block_2': 0.4, 'dropout_cnn_block_3': 0.30000000000000004, 'dropout_cnn_block_4': 0.6000000000000001, 'dropout_cnn_block_5': 0.4}
accuracy=0.9399092793464661
{'batch_size': 30.0, 'dropout_cnn_block_1': 0.4, 'dropout_cnn_block_2': 0.5, 'dropout_cnn_block_3': 0.4, 'dropout_cnn_block_4': 0.5, 'dropout_cnn_block_5': 0.6000000000000001}
accuracy=0.8408163189888
{'batch_size': 90.0, 'dropout_cnn_block_1': 0.5, 'dropout_cnn_block_2': 0.4, 'dropout_cnn_block_3': 0.4, 'dropout_cnn_block_4': 0.5, 'dropout_cnn_block_5': 0.6000000000000001}
accuracy=0.9247165322303772
{'batch_size': 60.0, 'dropout_cnn_block_1': 0.5, 'dropout_cnn_block_2': 0.30000000000000004, 'dropout_cnn_block_3': 0.4, 'dropout_cnn_bloc